In [53]:
from __future__ import print_function, division

from itertools import starmap, product

import numpy as np
import pandas as pd

import pickle

In [2]:
def obtainable(coins):
    combos = set(coins)
    combos.update(map(sum, product(coins, coins)))
    return combos

In [3]:
coins = [1, 2, 4]

In [4]:
obtainable(coins)

{1, 2, 3, 4, 5, 6, 8}

In [5]:
values = set(range(1, 11))
    
def unobtainable(coins):
    return values - obtainable(coins)

In [6]:
unobtainable(coins)

{7, 9, 10}

In [7]:
def possible_new_coin(coins, value):
    if value % 2 == 0:
        yield value // 2
        
    for coin in coins:
        if value > coin:
            yield value - coin

In [8]:
set(possible_new_coin(coins, 7))

{3, 5, 6}

In [9]:
set(possible_new_coin(coins, 9))

{5, 7, 8}

In [10]:
set(possible_new_coin(coins, 10))

{5, 6, 8, 9}

In [11]:
class Combinator:
    def __init__(self, n):
        self.values = set(range(1, n+1))
        self.combos = set()
        self.considered = set()
        self.smallest = 9999

    def unobtainable(self, coins):
        return self.values - obtainable(coins)

    def add_combo(self, coins):
        n = len(coins)
        if n < self.smallest:
            self.combos = set()
            self.smallest = n
            print('new smallest = ', n)
        self.combos.add(coins)

    def consider(self, coins):
        if len(coins) > self.smallest:
            return
        
        if coins in self.considered:
            return

        if len(coins) < 10:
            self.considered.add(coins)
        
        bad_values = self.unobtainable(coins)
        if len(bad_values) == 0:
            self.add_combo(coins)
            return
    
        for value in sorted(bad_values):
            for new_coin in sorted(possible_new_coin(coins, value)):
                self.consider(coins | {new_coin})
                
    def winners(self):
        for combo in self.combos:
            if len(combo) == self.smallest:
                yield combo

In [12]:
cmap = {}

In [13]:
cmap[10] = Combinator(10)
cmap[10].values

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10}

In [14]:
cmap[10].consider(frozenset())

new smallest =  5
new smallest =  4


In [15]:
cmap[10].smallest

4

In [16]:
for winner in cmap[10].winners():
    print(sorted(winner))

[1, 3, 5, 6]
[1, 2, 3, 7]
[1, 2, 5, 8]
[1, 3, 4, 9]
[1, 3, 4, 5]
[1, 2, 5, 7]
[1, 2, 4, 5]
[1, 3, 4, 6]


In [17]:
cmap[15] = Combinator(15)
cmap[15].values

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15}

In [18]:
for winner in cmap[10].winners():
    cmap[15].consider(winner)

new smallest =  6
new smallest =  5


In [19]:
cmap[15].consider(frozenset())

In [20]:
for winner in cmap[15].winners():
    print(sorted(winner))

[1, 3, 5, 7, 8]
[1, 3, 4, 9, 11]


In [21]:
def upgrade(cmap, source, dest):
    cmap[dest] = Combinator(dest)

    for winner in cmap[source].winners():
        cmap[dest].consider(winner)

    if cmap[dest].smallest > cmap[source].smallest:
        print('considering 1')
        cmap[dest].consider(frozenset((1, 2, 5)))
        print('considering 2')
        cmap[dest].consider(frozenset((1, 2, 4)))
        print('considering 3')
        cmap[dest].consider(frozenset((1, 3, 4)))
        print('considering 4')
        cmap[dest].consider(frozenset((1, 3, 5)))

    cmap[dest].considered = set()
    for winner in cmap[dest].winners():
        print(sorted(winner))

In [22]:
cmap = {}

In [23]:
cmap[10] = Combinator(10)
cmap[10].consider(frozenset())

new smallest =  5
new smallest =  4


In [24]:
for n in range(10, 20):
    print(n+1)
    %time upgrade(cmap, n, n+1)

11
new smallest =  4
[1, 3, 5, 6]
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 301 µs
12
new smallest =  4
[1, 3, 5, 6]
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 96.3 µs
13
new smallest =  5
considering 1
considering 2
considering 3
considering 4
[1, 3, 5, 7, 8]
[1, 3, 5, 6, 8]
[1, 2, 5, 6, 8]
[1, 2, 5, 8, 11]
[1, 3, 5, 6, 10]
[1, 2, 4, 6, 7]
[1, 2, 4, 6, 9]
[1, 2, 3, 4, 9]
[1, 3, 4, 5, 8]
[1, 3, 5, 6, 7]
[1, 2, 5, 8, 10]
[1, 3, 4, 9, 10]
[1, 3, 4, 8, 9]
[1, 3, 4, 9, 11]
[1, 2, 4, 5, 11]
[1, 3, 4, 6, 7]
[1, 3, 4, 8, 10]
[1, 3, 4, 6, 10]
[1, 2, 5, 6, 7]
[1, 3, 4, 7, 9]
[1, 3, 5, 6, 12]
[1, 2, 5, 7, 11]
CPU times: user 8 ms, sys: 0 ns, total: 8 ms
Wall time: 5.88 ms
14
new smallest =  5
[1, 2, 5, 7, 11]
[1, 3, 5, 7, 8]
[1, 3, 4, 9, 11]
[1, 3, 5, 6, 8]
[1, 3, 4, 6, 7]
[1, 2, 5, 6, 8]
[1, 3, 4, 8, 10]
[1, 3, 4, 6, 10]
[1, 3, 4, 7, 9]
[1, 3, 5, 6, 7]
[1, 2, 4, 6, 7]
[1, 3, 4, 9, 10]
[1, 2, 5, 6, 7]
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 783 µs
15
ne

In [25]:
for n in range(20, 25):
    print(n+1)
    %time upgrade(cmap, n, n+1)

21
new smallest =  7
considering 1
considering 2
considering 3
considering 4
[1, 3, 4, 7, 9, 12, 13]
[1, 2, 3, 4, 7, 12, 17]
[1, 2, 4, 5, 11, 13, 19]
[1, 2, 5, 8, 10, 13, 17]
[1, 2, 3, 4, 8, 13, 16]
[1, 2, 4, 5, 6, 13, 15]
[1, 3, 5, 6, 7, 9, 14]
[1, 2, 4, 7, 9, 10, 11]
[1, 2, 4, 7, 9, 12, 13]
[1, 3, 4, 7, 9, 12, 16]
[1, 3, 5, 6, 11, 13, 15]
[1, 2, 5, 7, 10, 11, 19]
[1, 3, 5, 6, 10, 11, 18]
[1, 3, 4, 7, 8, 9, 12]
[1, 3, 5, 7, 9, 10, 20]
[1, 3, 4, 7, 8, 12, 17]
[1, 2, 4, 7, 8, 11, 13]
[1, 3, 4, 9, 11, 12, 16]
[1, 3, 4, 6, 7, 15, 16]
[1, 3, 5, 7, 8, 12, 18]
[1, 2, 5, 6, 9, 10, 12]
[1, 3, 4, 9, 11, 16, 20]
[1, 3, 5, 6, 13, 14, 20]
[1, 2, 3, 4, 9, 12, 17]
[1, 2, 4, 5, 8, 10, 17]
[1, 2, 5, 7, 8, 10, 14]
[1, 2, 4, 7, 8, 13, 17]
[1, 3, 4, 9, 10, 12, 14]
[1, 3, 5, 6, 10, 14, 18]
[1, 3, 4, 5, 8, 13, 15]
[1, 3, 5, 7, 8, 16, 17]
[1, 3, 4, 9, 10, 12, 13]
[1, 2, 5, 7, 10, 11, 12]
[1, 2, 4, 7, 9, 10, 14]
[1, 3, 4, 5, 8, 14, 16]
[1, 2, 5, 7, 8, 9, 19]
[1, 3, 4, 5, 9, 10, 16]
[1, 2, 5, 8, 10, 12, 19]
[

In [26]:
for n in range(25, 30):
    print(n+1)
    %time upgrade(cmap, n, n+1)

26
new smallest =  8
new smallest =  7
[1, 2, 5, 8, 11, 12, 13]
[1, 3, 4, 9, 10, 12, 13]
[1, 3, 5, 7, 8, 17, 18]
CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 332 µs
27
new smallest =  8
considering 1
considering 2
considering 3
considering 4
[1, 3, 4, 9, 11, 12, 16, 17]
[1, 2, 5, 7, 8, 11, 16, 18]
[1, 2, 5, 6, 7, 8, 17, 19]
[1, 2, 5, 7, 10, 11, 14, 16]
[1, 2, 3, 4, 9, 13, 18, 23]
[1, 3, 4, 9, 11, 13, 16, 20]
[1, 3, 4, 9, 11, 15, 16, 20]
[1, 3, 5, 6, 10, 13, 16, 24]
[1, 3, 4, 5, 11, 13, 14, 20]
[1, 3, 5, 6, 8, 10, 17, 18]
[1, 3, 5, 6, 8, 9, 18, 19]
[1, 3, 4, 6, 9, 11, 16, 20]
[1, 3, 4, 8, 10, 11, 17, 23]
[1, 2, 4, 5, 9, 12, 15, 21]
[1, 2, 4, 7, 8, 13, 18, 23]
[1, 2, 5, 6, 7, 14, 17, 25]
[1, 2, 3, 4, 9, 13, 18, 21]
[1, 2, 5, 8, 11, 13, 14, 15]
[1, 2, 5, 7, 10, 13, 14, 15]
[1, 3, 4, 9, 10, 11, 16, 23]
[1, 3, 4, 8, 9, 11, 13, 14]
[1, 2, 5, 6, 8, 13, 17, 19]
[1, 3, 4, 6, 11, 13, 14, 20]
[1, 3, 5, 6, 7, 11, 12, 20]
[1, 3, 4, 9, 11, 16, 20, 25]
[1, 2, 4, 7, 9, 12, 13, 23]
[1, 2, 5,

In [27]:
n = 30
print(n+1)
%time upgrade(cmap, n, n+1)

31
new smallest =  9
new smallest =  8
[1, 2, 5, 8, 11, 14, 15, 16]
[1, 3, 4, 5, 8, 14, 20, 26]
[1, 3, 5, 7, 9, 10, 21, 22]
[1, 3, 5, 7, 9, 10, 20, 21]
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 811 µs


In [28]:
n = 31
print(n+1)
%time upgrade(cmap, n, n+1)

32
new smallest =  8
[1, 2, 5, 8, 11, 14, 15, 16]
[1, 3, 5, 7, 9, 10, 21, 22]
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 190 µs


In [29]:
n = 32
print(n+1)
%time upgrade(cmap, n, n+1)

33
new smallest =  9
considering 1
considering 2
considering 3
considering 4
[1, 2, 5, 8, 9, 10, 18, 20, 23]
[1, 3, 5, 6, 13, 15, 16, 20, 21]
[1, 3, 5, 6, 13, 14, 16, 17, 19]
[1, 3, 4, 5, 8, 14, 17, 23, 29]
[1, 2, 4, 6, 8, 10, 11, 22, 25]
[1, 3, 4, 9, 11, 16, 17, 20, 26]
[1, 3, 4, 6, 10, 14, 15, 16, 17]
[1, 3, 4, 6, 8, 9, 11, 21, 22]
[1, 3, 5, 7, 9, 10, 20, 21, 32]
[1, 2, 5, 7, 10, 13, 15, 19, 26]
[1, 3, 5, 6, 11, 12, 17, 19, 21]
[1, 2, 5, 6, 7, 11, 15, 18, 26]
[1, 3, 4, 9, 11, 12, 14, 16, 17]
[1, 3, 4, 5, 8, 14, 20, 23, 29]
[1, 3, 4, 5, 11, 13, 18, 20, 27]
[1, 3, 5, 6, 8, 11, 15, 21, 25]
[1, 3, 5, 6, 8, 14, 16, 17, 26]
[1, 3, 5, 6, 10, 11, 18, 20, 27]
[1, 2, 4, 5, 6, 11, 13, 19, 27]
[1, 2, 5, 8, 10, 14, 16, 19, 23]
[1, 2, 5, 8, 10, 13, 14, 15, 31]
[1, 3, 4, 6, 10, 13, 15, 18, 26]
[1, 3, 4, 9, 11, 16, 19, 20, 22]
[1, 3, 4, 8, 9, 14, 15, 17, 19]
[1, 3, 4, 9, 10, 13, 15, 18, 29]
[1, 3, 4, 9, 11, 12, 14, 18, 19]
[1, 3, 4, 9, 11, 16, 17, 20, 29]
[1, 3, 4, 7, 8, 13, 15, 23, 25]
[1, 2, 5, 6,

In [30]:
n = 33
print(n+1)
%time upgrade(cmap, n, n+1)

34
new smallest =  10
new smallest =  9
[1, 3, 5, 6, 13, 15, 16, 20, 21]
[1, 3, 5, 6, 13, 14, 16, 17, 19]
[1, 3, 4, 5, 8, 14, 17, 23, 29]
[1, 3, 4, 9, 11, 16, 17, 20, 26]
[1, 3, 4, 6, 10, 14, 15, 16, 17]
[1, 2, 5, 7, 10, 13, 15, 19, 26]
[1, 3, 5, 6, 11, 12, 17, 19, 21]
[1, 3, 4, 9, 11, 12, 14, 16, 17]
[1, 3, 4, 5, 8, 14, 20, 23, 29]
[1, 3, 5, 6, 8, 14, 16, 17, 26]
[1, 3, 4, 8, 9, 14, 15, 17, 19]
[1, 3, 4, 9, 11, 16, 17, 20, 29]
[1, 2, 5, 6, 8, 14, 16, 17, 21]
[1, 3, 4, 9, 10, 14, 16, 17, 19]
[1, 2, 4, 5, 11, 13, 14, 19, 29]
[1, 3, 4, 9, 11, 14, 15, 18, 30]
[1, 3, 4, 8, 9, 14, 16, 17, 26]
[1, 3, 5, 6, 10, 14, 15, 16, 17]
[1, 3, 4, 6, 10, 15, 16, 18, 23]
[1, 2, 5, 6, 9, 10, 12, 22, 24]
[1, 3, 4, 9, 11, 16, 17, 20, 27]
[1, 2, 4, 5, 10, 12, 17, 21, 28]
[1, 3, 4, 7, 8, 11, 13, 23, 25]
[1, 3, 4, 7, 9, 14, 15, 17, 18]
[1, 2, 5, 8, 11, 14, 17, 18, 19]
[1, 3, 5, 7, 8, 14, 15, 24, 26]
[1, 3, 4, 9, 10, 13, 14, 16, 17]
[1, 3, 4, 8, 9, 11, 20, 22, 23]
[1, 3, 4, 9, 11, 12, 15, 16, 17]
[1, 3, 5, 6, 8

In [31]:
n = 34
print(n+1)
%time upgrade(cmap, n, n+1)

35
new smallest =  9
[1, 3, 4, 7, 8, 10, 11, 23, 25]
[1, 3, 4, 5, 8, 14, 20, 26, 32]
[1, 3, 5, 6, 12, 13, 17, 18, 27]
[1, 3, 5, 6, 13, 15, 16, 20, 21]
[1, 3, 5, 6, 13, 14, 16, 17, 19]
[1, 3, 4, 6, 11, 13, 14, 20, 29]
[1, 3, 4, 9, 11, 16, 17, 20, 26]
[1, 2, 5, 6, 7, 15, 18, 26, 28]
[1, 3, 4, 9, 11, 15, 16, 18, 19]
[1, 3, 4, 7, 9, 15, 17, 20, 24]
[1, 2, 5, 8, 11, 14, 15, 16, 19]
[1, 3, 4, 6, 7, 10, 15, 23, 28]
[1, 3, 5, 6, 13, 15, 17, 24, 30]
[1, 3, 4, 5, 6, 9, 16, 23, 30]
[1, 3, 4, 9, 10, 14, 15, 17, 18]
[1, 2, 5, 7, 11, 13, 16, 18, 28]
[1, 2, 5, 6, 8, 14, 16, 17, 21]
[1, 3, 4, 9, 11, 15, 17, 22, 26]
[1, 3, 4, 9, 10, 14, 16, 17, 19]
[1, 3, 5, 6, 12, 13, 15, 17, 18]
[1, 3, 5, 6, 7, 12, 15, 20, 28]
[1, 2, 5, 8, 9, 10, 20, 22, 25]
[1, 2, 5, 8, 9, 10, 21, 24, 27]
[1, 3, 4, 9, 10, 13, 14, 20, 22]
[1, 3, 4, 8, 9, 14, 16, 17, 26]
[1, 2, 5, 6, 9, 10, 11, 23, 25]
[1, 3, 5, 7, 8, 17, 18, 26, 27]
[1, 3, 5, 6, 11, 13, 14, 20, 29]
[1, 3, 5, 6, 13, 14, 16, 18, 19]
[1, 3, 4, 8, 10, 15, 17, 18, 21]
[1,

In [32]:
n = 35
print(n+1)
%time upgrade(cmap, n, n+1)

36
new smallest =  9
[1, 3, 4, 7, 8, 10, 11, 23, 25]
[1, 3, 4, 9, 10, 15, 16, 17, 19]
[1, 3, 4, 5, 8, 14, 20, 26, 32]
[1, 3, 5, 6, 12, 13, 17, 18, 27]
[1, 3, 5, 6, 12, 13, 15, 17, 30]
[1, 2, 5, 7, 11, 15, 17, 18, 20]
[1, 2, 5, 7, 11, 14, 16, 19, 29]
[1, 3, 4, 8, 10, 15, 17, 18, 21]
[1, 3, 5, 6, 13, 15, 16, 20, 21]
[1, 3, 5, 6, 13, 14, 16, 17, 19]
[1, 2, 4, 7, 10, 13, 16, 17, 18]
[1, 3, 4, 7, 9, 14, 15, 17, 18]
[1, 3, 4, 9, 11, 16, 17, 20, 26]
[1, 2, 5, 6, 7, 15, 18, 26, 28]
[1, 2, 5, 8, 11, 14, 17, 18, 19]
[1, 3, 4, 9, 11, 15, 16, 18, 19]
[1, 2, 5, 8, 10, 13, 17, 23, 27]
[1, 3, 5, 6, 8, 11, 15, 24, 28]
[1, 2, 5, 7, 11, 13, 16, 17, 18]
[1, 3, 4, 8, 10, 15, 17, 21, 25]
[1, 3, 4, 8, 10, 14, 16, 17, 19]
[1, 3, 4, 6, 10, 13, 15, 21, 29]
[1, 2, 5, 7, 11, 15, 16, 17, 18]
[1, 2, 5, 8, 10, 13, 16, 17, 18]
[1, 3, 5, 6, 13, 15, 17, 18, 24]
[1, 3, 5, 7, 9, 11, 12, 23, 24]
[1, 3, 4, 9, 10, 15, 16, 18, 19]
[1, 3, 4, 7, 9, 14, 17, 18, 29]
[1, 2, 5, 8, 11, 14, 17, 19, 21]
[1, 3, 4, 8, 9, 14, 15, 17, 1

In [33]:
n = 36
print(n+1)
%time upgrade(cmap, n, n+1)

37
new smallest =  10
new smallest =  9
[1, 3, 5, 7, 9, 11, 12, 24, 25]
[1, 3, 5, 6, 13, 15, 17, 24, 30]
[1, 3, 4, 5, 8, 14, 20, 26, 32]
[1, 2, 5, 8, 9, 10, 21, 24, 27]
[1, 2, 5, 7, 11, 15, 17, 18, 20]
[1, 3, 4, 8, 10, 15, 16, 18, 19]
[1, 3, 5, 6, 13, 15, 16, 20, 21]
[1, 3, 4, 9, 11, 15, 16, 17, 20]
[1, 3, 4, 9, 11, 15, 17, 18, 22]
[1, 3, 5, 6, 13, 15, 16, 18, 22]
[1, 3, 4, 9, 11, 16, 17, 20, 26]
[1, 2, 5, 8, 11, 14, 17, 18, 19]
[1, 2, 5, 8, 10, 13, 17, 23, 27]
[1, 3, 4, 9, 11, 15, 16, 18, 19]
[1, 3, 4, 9, 11, 16, 17, 19, 20]
[1, 3, 5, 7, 9, 11, 12, 25, 26]
[1, 3, 5, 6, 13, 14, 18, 19, 29]
[1, 3, 5, 6, 13, 14, 16, 18, 19]
[1, 3, 4, 9, 10, 15, 16, 18, 19]
[1, 3, 5, 6, 13, 15, 17, 18, 24]
CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 2.88 ms


In [34]:
n = 37
print(n+1)
%time upgrade(cmap, n, n+1)

38
new smallest =  10
new smallest =  9
[1, 2, 5, 7, 11, 15, 17, 18, 20]
[1, 3, 4, 8, 10, 15, 16, 18, 19]
[1, 3, 4, 9, 10, 15, 16, 18, 19]
[1, 2, 5, 8, 11, 14, 17, 18, 19]
[1, 3, 4, 9, 11, 15, 16, 18, 19]
[1, 3, 5, 6, 13, 14, 18, 19, 29]
[1, 3, 5, 7, 9, 11, 12, 25, 26]
[1, 3, 4, 9, 11, 16, 17, 19, 20]
[1, 3, 5, 6, 13, 14, 16, 18, 19]
[1, 3, 5, 6, 13, 15, 16, 18, 22]
CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 2.88 ms


In [35]:
n = 38
print(n+1)
%time upgrade(cmap, n, n+1)

39
new smallest =  10
new smallest =  9
[1, 3, 4, 9, 11, 16, 17, 19, 20]
CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 3.24 ms


In [36]:
n = 39
print(n+1)
%time upgrade(cmap, n, n+1)

40
new smallest =  9
[1, 3, 4, 9, 11, 16, 17, 19, 20]
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 140 µs


In [37]:
n = 40
print(n+1)
%time upgrade(cmap, n, n+1)

41
new smallest =  10
considering 1
considering 2
considering 3
considering 4
[1, 3, 4, 9, 11, 16, 18, 19, 21, 22]
[1, 2, 5, 8, 11, 12, 13, 24, 26, 28]
[1, 3, 5, 6, 11, 12, 19, 21, 23, 36]
[1, 2, 4, 5, 6, 7, 11, 19, 27, 35]
[1, 3, 5, 7, 9, 10, 19, 20, 28, 31]
[1, 3, 4, 8, 10, 14, 18, 19, 22, 31]
[1, 3, 4, 8, 10, 14, 19, 20, 22, 27]
[1, 3, 5, 6, 11, 12, 16, 19, 25, 28]
[1, 2, 5, 7, 11, 13, 17, 18, 20, 21]
[1, 3, 4, 7, 8, 10, 19, 21, 29, 31]
[1, 3, 5, 6, 12, 14, 15, 19, 20, 36]
[1, 3, 4, 9, 10, 12, 13, 27, 29, 34]
[1, 3, 5, 7, 9, 10, 21, 22, 32, 33]
[1, 3, 4, 9, 11, 15, 17, 20, 21, 24]
[1, 3, 4, 9, 11, 16, 17, 19, 20, 22]
[1, 3, 5, 6, 13, 14, 15, 19, 22, 26]
[1, 3, 4, 7, 8, 11, 12, 14, 27, 29]
[1, 3, 5, 6, 9, 13, 17, 19, 20, 22]
[1, 2, 5, 8, 10, 11, 13, 17, 27, 31]
[1, 2, 3, 5, 9, 13, 17, 19, 20, 22]
[1, 3, 5, 6, 13, 15, 17, 18, 20, 26]
[1, 3, 4, 8, 9, 11, 16, 20, 26, 30]
[1, 2, 5, 7, 11, 15, 17, 20, 22, 36]
[1, 3, 5, 6, 7, 10, 12, 18, 26, 34]
[1, 2, 5, 7, 10, 13, 15, 19, 26, 30]
[1, 2, 

In [39]:
n = 41
print(n+1)
%time upgrade(cmap, n, n+1)

42
new smallest =  10
[1, 3, 4, 9, 11, 16, 18, 19, 21, 22]
[1, 3, 5, 6, 11, 12, 19, 21, 23, 36]
[1, 3, 5, 7, 9, 10, 20, 21, 32, 33]
[1, 3, 4, 6, 10, 13, 15, 21, 29, 37]
[1, 2, 4, 5, 6, 7, 11, 19, 27, 35]
[1, 3, 5, 7, 8, 12, 14, 18, 26, 34]
[1, 3, 4, 9, 11, 13, 18, 19, 21, 22]
[1, 3, 4, 9, 10, 12, 13, 24, 29, 31]
[1, 3, 4, 8, 10, 14, 19, 20, 22, 27]
[1, 2, 5, 8, 10, 13, 16, 19, 20, 21]
[1, 3, 4, 9, 10, 13, 14, 21, 27, 29]
[1, 2, 5, 6, 8, 9, 19, 21, 30, 32]
[1, 3, 5, 7, 9, 10, 19, 20, 31, 32]
[1, 3, 4, 8, 10, 11, 13, 24, 28, 29]
[1, 2, 5, 7, 11, 13, 17, 18, 20, 21]
[1, 3, 4, 7, 8, 10, 19, 21, 29, 31]
[1, 3, 4, 9, 11, 16, 19, 20, 25, 33]
[1, 3, 4, 6, 10, 12, 17, 18, 20, 21]
[1, 3, 5, 6, 12, 13, 18, 19, 27, 29]
[1, 3, 4, 9, 10, 12, 13, 23, 28, 30]
[1, 3, 5, 6, 13, 15, 16, 19, 20, 21]
[1, 3, 4, 9, 11, 15, 17, 18, 20, 24]
[1, 3, 5, 6, 12, 14, 15, 19, 20, 36]
[1, 2, 5, 7, 10, 13, 16, 19, 20, 21]
[1, 2, 4, 5, 8, 10, 17, 20, 21, 31]
[1, 3, 4, 9, 11, 12, 16, 26, 30, 36]
[1, 3, 4, 6, 10, 14, 18, 

In [40]:
n = 42
print(n+1)
%time upgrade(cmap, n, n+1)

43
new smallest =  10
[1, 3, 4, 9, 11, 16, 18, 19, 21, 22]
[1, 3, 4, 9, 11, 13, 18, 19, 21, 32]
[1, 3, 5, 7, 9, 10, 20, 21, 32, 33]
[1, 3, 4, 6, 10, 13, 15, 21, 29, 37]
[1, 3, 4, 9, 11, 13, 18, 19, 21, 22]
[1, 3, 4, 5, 6, 9, 16, 23, 30, 37]
[1, 3, 5, 6, 13, 15, 17, 20, 21, 26]
[1, 3, 4, 9, 10, 12, 13, 24, 29, 31]
[1, 3, 5, 6, 13, 15, 17, 18, 24, 25]
[1, 3, 4, 9, 10, 13, 14, 21, 27, 29]
[1, 3, 4, 9, 10, 12, 13, 26, 28, 33]
[1, 2, 5, 7, 11, 15, 19, 20, 21, 22]
[1, 3, 5, 7, 9, 10, 21, 22, 33, 34]
[1, 2, 5, 8, 11, 12, 13, 27, 29, 31]
[1, 2, 5, 7, 11, 15, 17, 20, 22, 36]
[1, 3, 5, 7, 9, 10, 21, 22, 31, 32]
[1, 3, 5, 6, 10, 14, 18, 20, 21, 23]
[1, 3, 5, 6, 13, 15, 17, 18, 24, 37]
[1, 3, 4, 9, 10, 12, 13, 26, 28, 30]
[1, 3, 5, 7, 9, 10, 21, 22, 30, 31]
[1, 3, 4, 9, 10, 12, 13, 23, 28, 30]
[1, 3, 4, 8, 9, 10, 14, 17, 29, 32]
[1, 3, 5, 7, 9, 11, 13, 14, 28, 29]
[1, 2, 5, 8, 11, 12, 13, 26, 28, 30]
[1, 2, 5, 7, 11, 15, 19, 21, 24, 26]
[1, 3, 4, 9, 11, 16, 19, 20, 22, 34]
[1, 2, 5, 7, 11, 15, 19,

In [41]:
n = 43
print(n+1)
%time upgrade(cmap, n, n+1)

44
new smallest =  10
[1, 3, 4, 9, 11, 16, 18, 19, 21, 22]
[1, 3, 4, 6, 10, 13, 15, 21, 29, 37]
[1, 3, 4, 9, 11, 13, 18, 19, 21, 22]
[1, 3, 4, 9, 10, 12, 13, 24, 29, 31]
[1, 2, 5, 7, 11, 15, 19, 20, 21, 22]
[1, 3, 5, 7, 9, 10, 21, 22, 33, 34]
[1, 2, 5, 8, 11, 12, 13, 27, 29, 31]
[1, 2, 5, 7, 11, 15, 17, 20, 22, 36]
[1, 3, 5, 7, 9, 10, 21, 22, 31, 32]
[1, 3, 5, 6, 10, 14, 18, 20, 21, 23]
[1, 3, 5, 7, 9, 10, 21, 22, 30, 31]
[1, 3, 4, 9, 11, 16, 19, 20, 22, 34]
[1, 2, 5, 7, 11, 15, 19, 21, 22, 24]
[1, 3, 4, 6, 10, 14, 18, 20, 21, 23]
[1, 2, 5, 7, 11, 15, 17, 20, 21, 22]
[1, 3, 4, 9, 10, 12, 13, 27, 29, 34]
[1, 2, 5, 8, 11, 14, 17, 20, 21, 22]
[1, 3, 4, 6, 11, 13, 18, 19, 21, 22]
[1, 3, 5, 7, 9, 10, 21, 22, 32, 33]
[1, 3, 5, 6, 12, 13, 19, 20, 22, 24]
[1, 3, 4, 8, 10, 14, 18, 20, 21, 23]
[1, 3, 5, 6, 13, 15, 16, 20, 21, 38]
[1, 3, 5, 6, 13, 15, 16, 20, 21, 23]
[1, 3, 5, 7, 9, 11, 13, 14, 29, 30]
[1, 3, 4, 9, 10, 12, 13, 27, 29, 31]
CPU times: user 8 ms, sys: 0 ns, total: 8 ms
Wall time: 5.

In [42]:
n = 44
print(n+1)
%time upgrade(cmap, n, n+1)

45
new smallest =  11
new smallest =  10
[1, 3, 4, 9, 11, 16, 19, 20, 22, 34]
[1, 2, 5, 7, 11, 15, 19, 21, 22, 24]
CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 6.3 ms


In [43]:
n = 45
print(n+1)
%time upgrade(cmap, n, n+1)

46
new smallest =  11
new smallest =  10
[1, 2, 5, 7, 11, 15, 19, 21, 22, 24]
CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 726 µs


In [ ]:
n = 46
print(n+1)
%time upgrade(cmap, n, n+1)

In [51]:
pickle.dump(cmap, open( "coins.p", "wb" ) )

In [52]:
pickle.load( open( "coins.p", "rb" ) )

{10: <__main__.Combinator at 0x7fd57c8802b0>,
 11: <__main__.Combinator at 0x7fd57c880390>,
 12: <__main__.Combinator at 0x7fd57c8803c8>,
 13: <__main__.Combinator at 0x7fd57c880400>,
 14: <__main__.Combinator at 0x7fd57c880438>,
 15: <__main__.Combinator at 0x7fd57c880470>,
 16: <__main__.Combinator at 0x7fd57c8804a8>,
 17: <__main__.Combinator at 0x7fd57c8804e0>,
 18: <__main__.Combinator at 0x7fd57c880518>,
 19: <__main__.Combinator at 0x7fd57c880550>,
 20: <__main__.Combinator at 0x7fd57c880588>,
 21: <__main__.Combinator at 0x7fd57c8805c0>,
 22: <__main__.Combinator at 0x7fd57c880630>,
 23: <__main__.Combinator at 0x7fd57c880668>,
 24: <__main__.Combinator at 0x7fd57c8806a0>,
 25: <__main__.Combinator at 0x7fd57c8806d8>,
 26: <__main__.Combinator at 0x7fd57c880128>,
 27: <__main__.Combinator at 0x7fd57c880710>,
 28: <__main__.Combinator at 0x7fd57c880748>,
 29: <__main__.Combinator at 0x7fd57c880780>,
 30: <__main__.Combinator at 0x7fd57c8807b8>,
 31: <__main__.Combinator at 0x7fd